In [1]:
import numpy as np
import pandas as pd

from string import punctuation

import spacy
from spacy.lang.en.stop_words import STOP_WORDS

from collections import Counter

In [2]:
data = pd.read_csv('../Datasets/train-balanced-sarcasm.csv')

In [3]:
data.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010826 entries, 0 to 1010825
Data columns (total 10 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   label           1010826 non-null  int64 
 1   comment         1010773 non-null  object
 2   author          1010826 non-null  object
 3   subreddit       1010826 non-null  object
 4   score           1010826 non-null  int64 
 5   ups             1010826 non-null  int64 
 6   downs           1010826 non-null  int64 
 7   date            1010826 non-null  object
 8   created_utc     1010826 non-null  object
 9   parent_comment  1010826 non-null  object
dtypes: int64(4), object(6)
memory usage: 77.1+ MB


In [5]:
data['date'] = pd.to_datetime(data['date']).dt.date

In [6]:
data.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10-01,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11-01,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09-01,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10-01,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12-01,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


In [7]:
data['date'].min(), data['date'].max()

(datetime.date(2009, 1, 1), datetime.date(2016, 12, 1))

In [8]:
data.date.value_counts()

2016-12-01    53292
2016-11-01    47989
2016-10-01    46161
2016-08-01    42197
2016-09-01    41152
              ...  
2009-04-01      122
2009-05-01      110
2009-03-01      105
2009-01-01      104
2009-02-01       89
Name: date, Length: 96, dtype: int64

In [9]:
recent_data = data.loc[pd.to_datetime(data.date).dt.year == 2016]

In [10]:
df = data[['comment', 'label']]

In [11]:
df['comment'] = df['comment'].str.lower()

<ipython-input-11-d730d9aacb9a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comment'] = df['comment'].str.lower()


In [12]:
df.fillna(' ', inplace=True)

/Users/doski/miniconda3/envs/getinno-env/lib/python3.9/site-packages/pandas/core/frame.py:5171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [13]:
df['comment_words'] = df['comment'].apply(lambda x: x.split(' '))

<ipython-input-13-71b550de0f8b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comment_words'] = df['comment'].apply(lambda x: x.split(' '))


In [14]:
df.head()

,comment,label,comment_words
0,nc and nh.,0,"[nc, and, nh.]"
1,you do know west teams play against west teams...,0,"[you, do, know, west, teams, play, against, we..."
2,"they were underdogs earlier today, but since g...",0,"[they, were, underdogs, earlier, today,, but, ..."
3,"this meme isn't funny none of the ""new york ni...",0,"[this, meme, isn't, funny, none, of, the, ""new..."
4,i could use one of those tools.,0,"[i, could, use, one, of, those, tools.]"


In [15]:
df['word_count'] = df['comment_words'].apply(len)

<ipython-input-15-dfcfab283c31>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['word_count'] = df['comment_words'].apply(len)


In [16]:
df.head()

,comment,label,comment_words,word_count
0,nc and nh.,0,"[nc, and, nh.]",3
1,you do know west teams play against west teams...,0,"[you, do, know, west, teams, play, against, we...",14
2,"they were underdogs earlier today, but since g...",0,"[they, were, underdogs, earlier, today,, but, ...",19
3,"this meme isn't funny none of the ""new york ni...",0,"[this, meme, isn't, funny, none, of, the, ""new...",12
4,i could use one of those tools.,0,"[i, could, use, one, of, those, tools.]",7


In [17]:
# Remove punctuations
df['comment_words'] = df['comment_words'].apply(
    lambda x: [word for word in x if word not in punctuation]
)

<ipython-input-17-a26680be395d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comment_words'] = df['comment_words'].apply(


In [18]:
# Remove stop words
df['comment_words'] = df['comment_words'].apply(
    lambda x: [word for word in x if (word not in STOP_WORDS)]
)

<ipython-input-18-12ff3feb71f0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comment_words'] = df['comment_words'].apply(


In [19]:
# Load english pre-trained spacy language model
en_model = spacy.load('en_core_web_sm', exclude=["tagger", "parser", "ner", "attribute_ruler"])
en_model.enable_pipe("lemmatizer")

In [20]:
import time

start = time.time()

In [21]:
# Extract the lemma from the words in the cleaned message
df['comment_lemma'] = df['comment_words'].apply(lambda x: [token.lemma_ for token in en_model(' '.join(x))])

/Users/doski/miniconda3/envs/getinno-env/lib/python3.9/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
<ipython-input-21-e6611625e7dc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comment_lemma'] = df['comment_words'].apply(lambda x: [token.lemma_ for token in en_model(' '.join(x))])


In [22]:
end = time.time() - start

In [23]:
end/60

28.156444998582206

In [24]:
start = time.time()

In [25]:
df['embeddings'] = df['comment'].apply(lambda x: [np.mean(token.vector) for token in en_model(x)])

<ipython-input-25-abcd1ebc6986>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['embeddings'] = df['comment'].apply(lambda x: [np.mean(token.vector) for token in en_model(x)])


In [26]:
end = time.time() - start

In [27]:
end/60

32.61304548581441

In [28]:
# Remove punctuations
df['comment_lemma'] = df['comment_lemma'].apply(
    lambda x: [word for word in x if word not in punctuation]
)

<ipython-input-28-ba622c8bacaf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comment_lemma'] = df['comment_lemma'].apply(


In [29]:
# Remove stop words
df['comment_lemma'] = df['comment_lemma'].apply(
    lambda x: [word for word in x if (word not in STOP_WORDS)]
)

<ipython-input-29-c12c84b4e63d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comment_lemma'] = df['comment_lemma'].apply(


In [30]:
df.head()

,comment,label,comment_words,word_count,comment_lemma,embeddings
0,nc and nh.,0,"[nc, nh.]",3,"[nc, nh]","[-0.015460032, 0.003731611, 0.020356858, 0.009..."
1,you do know west teams play against west teams...,0,"[know, west, teams, play, west, teams, east, t...",14,"[know, west, teams, play, west, teams, east, t...","[0.010391404, 0.03184216, -0.0034754674, -0.02..."
2,"they were underdogs earlier today, but since g...",0,"[underdogs, earlier, today,, gronk's, announce...",19,"[underdogs, earlier, today, gronk, announcemen...","[-0.014239922, -0.02061134, -0.005229493, 0.01..."
3,"this meme isn't funny none of the ""new york ni...",0,"[meme, isn't, funny, ""new, york, nigga"", ones,...",12,"[meme, funny, new, york, nigga, ones]","[-0.0061914944, -0.022155004, -0.0043228767, 0..."
4,i could use one of those tools.,0,"[use, tools.]",7,"[use, tools]","[-0.0146077005, 0.011123444, -0.013298005, -0...."


In [31]:
df['lemma'] = df['comment_lemma'].apply(lambda x: ' '.join(x))

<ipython-input-31-c0900dda70a0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lemma'] = df['comment_lemma'].apply(lambda x: ' '.join(x))


In [32]:
df[['lemma', 'label', 'comment', 'comment_words', 'embeddings']].to_csv('../Datasets/clean_reddit_data.csv', index=False)

In [33]:
# df.groupby(['label'], as_index=False).agg(
#     word_count_min = ('word_count', 'min'),
#     word_count_avg = ('word_count', 'mean'),
#     word_count_mid = ('word_count', 'median'),
#     word_count_max = ('word_count', 'max'),
#     common_words = ('comment_words', 'sum')
# )

In [34]:
# tmp = pd.DataFrame(df.comment_words.sum(), columns=['words'])

In [35]:
# tmp.value_counts(dropna=False)

In [36]:
# Counter(df.comment_words.sum()).most_common(100)